In [234]:
import numpy as np
import pandas as pd

In [235]:
df = pd.read_csv('train.csv')
df

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S
...,...,...,...,...,...,...,...,...,...,...,...,...
886,887,0,2,"Montvila, Rev. Juozas",male,27.0,0,0,211536,13.0000,NaN,S
887,888,1,1,"Graham, Miss. Margaret Edith",female,19.0,0,0,112053,30.0000,B42,S
888,889,0,3,"Johnston, Miss. Catherine Helen ""Carrie""",female,NaN,1,2,W./C. 6607,23.4500,NaN,S
889,890,1,1,"Behr, Mr. Karl Howell",male,26.0,0,0,111369,30.0000,C148,C


In [236]:
df.drop(columns=['PassengerId','Name','Ticket','Cabin'],inplace=True)

In [237]:
df

,Survived,Pclass,Sex,Age,SibSp,Parch,Fare,Embarked
0,0,3,male,22.0,1,0,7.2500,S
1,1,1,female,38.0,1,0,71.2833,C
2,1,3,female,26.0,0,0,7.9250,S
3,1,1,female,35.0,1,0,53.1000,S
4,0,3,male,35.0,0,0,8.0500,S
...,...,...,...,...,...,...,...,...
886,0,2,male,27.0,0,0,13.0000,S
887,1,1,female,19.0,0,0,30.0000,S
888,0,3,female,NaN,1,2,23.4500,S
889,1,1,male,26.0,0,0,30.0000,C


In [238]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(df.iloc[:,1:],df.iloc[:,0],test_size=0.2,random_state=0)

In [255]:
print(X_train.shape,X_test.shape,y_train.shape,y_test.shape)

(712, 7) (179, 7) (712,) (179,)


In [240]:
X_train.isnull().sum()

Pclass        0
Sex           0
Age         141
SibSp         0
Parch         0
Fare          0
Embarked      2
dtype: int64

In [241]:
X_train

,Pclass,Sex,Age,SibSp,Parch,Fare,Embarked
140,3,female,NaN,0,2,15.2458,C
439,2,male,31.0,0,0,10.5000,S
817,2,male,31.0,1,1,37.0042,C
378,3,male,20.0,0,0,4.0125,C
491,3,male,21.0,0,0,7.2500,S
...,...,...,...,...,...,...,...
835,1,female,39.0,1,1,83.1583,C
192,3,female,19.0,1,0,7.8542,S
629,3,male,NaN,0,0,7.7333,Q
559,3,female,36.0,1,0,17.4000,S


In [242]:
from sklearn.compose import ColumnTransformer
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import StandardScaler,OneHotEncoder
from sklearn.feature_selection import SelectKBest,chi2
from sklearn.tree import DecisionTreeClassifier
from sklearn.pipeline import Pipeline

# 1. imputation transformer

In [243]:
imput_trf = ColumnTransformer([
    ('num_impute',SimpleImputer(),[2]),
    ('cat_impute',SimpleImputer(strategy='most_frequent'),[-1]),
],remainder='passthrough')

# 2. Scaling transformer

In [244]:
scaling_trf = ColumnTransformer([
    ('num_scaling',StandardScaler(),[-2])
],remainder='passthrough')

# 3. Onehotencoding transformer

In [245]:
ohe_trf = ColumnTransformer([
    ('ohe_encoder',OneHotEncoder(sparse=False,drop='first'),[1,-1])
])

# Features selection

In [246]:
features_selection_trf = SelectKBest(score_func=chi2,k=8)

# Model train

In [247]:
train_model = DecisionTreeClassifier()

# Pipeline

In [248]:
pipe = Pipeline([
    ('impute_trf',imput_trf),
    ('scaling_trf',scaling_trf),
    ('ohe_trf',ohe_trf),
    ('features_selection_trf',features_selection_trf),
    ('train_model',train_model),
])

In [250]:
pipe.fit(X_train,y_train)

C:\Users\HP TECHNOLOGY\anaconda3\Lib\site-packages\sklearn\preprocessing\_encoders.py:868: FutureWarning: `sparse` was renamed to `sparse_output` in version 1.2 and will be removed in 1.4. `sparse_output` is ignored unless you leave `sparse` to its default value.
  warnings.warn(


Pipeline(steps=[('impute_trf',
                 ColumnTransformer(remainder='passthrough',
                                   transformers=[('num_impute', SimpleImputer(),
                                                  [2]),
                                                 ('cat_impute',
                                                  SimpleImputer(strategy='most_frequent'),
                                                  [-1])])),
                ('scaling_trf',
                 ColumnTransformer(remainder='passthrough',
                                   transformers=[('num_scaling',
                                                  StandardScaler(), [-2])])),
                ('ohe_trf',
                 ColumnTransformer(transformers=[('ohe_encoder',
                                                  OneHotEncoder(drop='first',
                                                                sparse=False),
                                                  [1, -1])])),
                ('features_selection_trf',
                 SelectKBest(k=8,
                             score_func=<function chi2 at 0x000001A6F34F93A0>)),
                ('train_model', DecisionTreeClassifier())])

In [253]:
y_pred = pipe.predict(X_test)

ValueError: Found unknown categories [0.42, 24.5] in column 0 during transform

In [254]:
from sklearn.metrics import accuracy_score
accuracy_score(y_test,y_pred)

NameError: name 'y_pred' is not defined